## Poker Player Patterns

The data needed is [here](https://www.kaggle.com/smeilz/poker-holdem-games#Export%20Holdem%20Manager%202.0%2012292016131233.txt), the specific file used is 
<b><span style="color: black; background-color: red;">Export Holdem Manager 2.0 12292016131233.txt</span></b>.

### Environment settings

In [70]:
from pathlib import Path
import pandas as pd

nrounds, nrowdisplay = 100, 10
pd.options.display.max_rows = nrowdisplay

### Loading csv

In [74]:
load_path = Path.cwd() / 'tidy_data'

action_ids = pd.read_csv(load_path / 'action_ids.csv')
actions = pd.read_csv(load_path / 'actions.csv')
cardshow = pd.read_csv(load_path / 'cardshow.csv')
buyins = pd.read_csv(load_path / 'buyins.csv')

action_dict = dict(zip(action_ids.action_name, action_ids.action_id))
actions = actions[actions.round_id <= nrounds]
cardshow = cardshow[cardshow.round_id <= nrounds]
buyins = buyins[buyins.round_id <= nrounds]

print(action_dict)

{'folds': 0, 'calls': 1, 'raises': 2, 'checks': 3, 'bets': 4, 'allin': 5}


### Basics
We'll gather some basic information about the players.
For each player we want:
- number of rounds played
- number of each action taken
- money put in each round
- money won out of pot
- relative winnings through the game

In [83]:
n_rounds = buyins.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        pd.DataFrame(
            data = {'rounds_played': [len(df.round_id)]}
        )
    )
).reset_index().drop(columns = ['level_1'])

In [84]:
n_actions = actions.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        data = {
            key: [len(df.action_id[df.action_id == i])]
            for key, i in action_dict.items()
        }
    )
).reset_index().drop(columns = ['level_1'])

In [85]:
winnings = cardshow.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        data = {
            'sum': [sum(df.amount)],
            'mean': [df.amount.mean()]
        }
    )
).reset_index().drop(columns = ['level_1'])

In [80]:
winnings

,user_id,folds,calls,raises,checks,bets,allin
0,0,3,1,2,0,2,0
1,1,13,0,4,0,2,1
2,2,64,4,11,5,4,2
3,3,14,1,1,0,2,0
4,4,14,3,2,1,0,0
...,...,...,...,...,...,...,...
28,28,7,2,0,1,1,0
29,29,5,6,3,4,1,0
30,30,7,2,3,1,0,0
31,31,5,1,0,1,1,0
